In [1]:
import numpy as np
import pandas as pd

In [2]:
temp_df = pd.read_csv('/home/manav-bassi/python_envs/data/IMDB Dataset.csv')

In [3]:
df = temp_df.iloc[10000:20000]

In [4]:
df.head()

,review,sentiment
10000,""" While sporadically engrossing (including a f...",negative
10001,"A French novelist, disgusted by his wife's soc...",positive
10002,"It must be remembered that the Gammera movies,...",negative
10003,"I gave this film 8 out of 10, reserving 10 for...",positive
10004,"Cheap, gloriously bad cheese from the 80's, th...",negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    5125
positive    4875
Name: count, dtype: int64

In [7]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
df.duplicated().sum()

0

In [9]:
df.drop_duplicates(inplace = True)

/tmp/ipykernel_5456/3424306917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace = True)


In [12]:
import re 
def remove_tags(text):
    cleaned_text = re.sub(re.compile('<.*?>'),'',text)
    return cleaned_text

In [13]:
df['review'] = df['review'].apply(remove_tags)

/tmp/ipykernel_5456/2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [14]:
df['review'] = df['review'].apply(lambda x : x.lower())

/tmp/ipykernel_5456/1265626467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x : x.lower())


In [18]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x: ' '.join(x))

[nltk_data] Downloading package stopwords to /home/manav-
[nltk_data]     bassi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipykernel_5456/2369742532.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x: ' '.join(x))


In [19]:
df.head()

,review,sentiment
10000,""" sporadically engrossing (including effective...",negative
10001,"french novelist, disgusted wife's society frie...",positive
10002,"must remembered gammera movies, like many firs...",negative
10003,"gave film 8 10, reserving 10 e.g amadeus, 9 sl...",positive
10004,"cheap, gloriously bad cheese 80's, decade chee...",negative


In [21]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [22]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
y = encode.fit_transform(y)

In [23]:
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 2)

In [24]:
X_train.shape

(7982, 1)

In [27]:
X_test.shape

(1996, 1)

In [28]:
# applying Bag of words 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

X_train_Bow = cv.fit_transform(X_train['review']).toarray()
X_test_Bow = cv.transform(X_test['review']).toarray()


In [30]:
X_train_Bow.shape

(7982, 47657)

In [31]:
# Naive_bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_Bow,y_train)

,priors,None
,var_smoothing,1e-09


In [32]:
y_pred = gnb.predict(X_test_Bow)

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [33]:
accuracy_score(y_pred,y_test)

0.6442885771543087

In [34]:
confusion_matrix(y_pred,y_test)

array([[781, 440],
       [270, 505]])

In [35]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.74      0.64      0.69      1221
           1       0.53      0.65      0.59       775

    accuracy                           0.64      1996
   macro avg       0.64      0.65      0.64      1996
weighted avg       0.66      0.64      0.65      1996



In [36]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

rfc.fit(X_train_Bow,y_train)
y_pred = rfc.predict(X_test_Bow)
accuracy_score(y_pred,y_test)

0.8401803607214429

In [38]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.86      0.84      0.85      1076
           1       0.82      0.84      0.83       920

    accuracy                           0.84      1996
   macro avg       0.84      0.84      0.84      1996
weighted avg       0.84      0.84      0.84      1996



In [39]:
confusion_matrix(y_pred,y_test)

array([[904, 172],
       [147, 773]])

In [40]:
# cv using ngrams

cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8381763527054108

In [41]:
# using TfIdf

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review']).toarray()

In [42]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

0.8401803607214429

In [43]:
import gensim

In [44]:
from gensim.models import Word2Vec,KeyedVectors

In [49]:
model  = KeyedVectors.load_word2vec_format('/home/manav-bassi/python_envs/data/GoogleNews-vectors-negative300.bin',binary = True,limit = 500000)

In [50]:
model['cricket'].shape

(300,)

In [59]:
import spacy
import en_core_web_sm
# load the spacy model
nlp = en_core_web_sm.load()
# X_train is a numpy nd.array and spacy model takes only str so convert it str
doc = nlp(str(X_train.values[0]))
print(doc.vector)

[-0.04809253 -0.26986128 -0.14906217 -0.14350408  0.03952795  0.13774733
  0.33070174  0.32989553  0.08226646 -0.05642227  0.32410333 -0.09346274
 -0.13367425 -0.00099252 -0.06156443  0.23211835 -0.17009929 -0.26110798
  0.10614458 -0.25056246 -0.1591415   0.3260067  -0.15154432 -0.034691
  0.1808421   0.05147072  0.09250318  0.36824512  0.05066487 -0.03923813
  0.11469033  0.01025553 -0.03937025 -0.03143087 -0.34891912  0.01038685
 -0.16010104 -0.00540249 -0.22199145 -0.13588893  0.04187847  0.11806451
 -0.06534873  0.12627602  0.06193534  0.29747412  0.01702496 -0.02829721
  0.21516126 -0.14398977  0.25913072  0.117359   -0.19223985 -0.16250622
 -0.00923703 -0.02998111  0.1372816  -0.00113597 -0.02925452  0.05382778
 -0.22036684  0.04664869 -0.15031217 -0.22268015  0.04566025 -0.00609082
  0.22758055 -0.20009156  0.08255083 -0.14644296  0.41995817  0.02336607
 -0.24920508 -0.2869741   0.11293471 -0.13387568 -0.07778857  0.01524858
  0.09639716 -0.3019998  -0.3580497   0.02427343 -0.1

In [ ]:
input_train = []
for item in X_train.values:
    doc = nlp(str(item))
    input_train.append(doc.vector)
input_train = np.array(input_train)

In [63]:
input_train.shape

(7982, 96)

In [65]:
input_test = []
for item in X_test.values:
    doc1 = nlp(str(item))
    input_test.append(doc1.vector)
input_test = np.array(input_test)

In [66]:
input_test.shape

(1996, 96)

In [67]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(input_train,y_train)
y_pred = gnb.predict(input_test)
accuracy_score(y_test,y_pred) 

0.6057114228456913